In [1]:
using JuMP
using MixedIntegerExperiments
using Polyhedra
using CDDLib
# using Plots
using DrakeVisualizer
using AxisArrayVariables
using AxisArrays
using BenchmarkTools
using StaticArrays
import BoxRobots

Loading cdd global constants


In [2]:
# plotlyjs();

In [3]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")
# Pkg.clone("git@github.com:JuliaPolyhedra/ConvexHull.jl.git")
# Pkg.checkout("Polyhedra")
# Pkg.checkout("CDDLib")

In [4]:
show_plots = false;

In [5]:
optparams = MIQPTrajOptParams(verbose = false)
contact_point_descriptions = Dict(:foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])))
robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
environment = [
    contact_region([0.; 0.], [1.; 0.], 0.8, 2 * robot.m * norm(robot.g)); 
    axis_aligned_free_box_region([0.; 0.], [1.; 1.])];

contactpointstates = Dict(:foot => ContactPointState(SVector(0.5, 0.)))
initialstate = BoxRobotWithRotation2DState(0., 0.1, SVector(0.5, 0.8), SVector(0.1, 0.), contactpointstates)
;

In [6]:
# if show_plots
#     display(plot_environment(environment))
#     for (i, region) in enumerate(environment)
#         isfree(region) || display(plot_allowable_forces(region, i))
#     end
#     display(plot_kinematic_regions(Dict(sym => limb.kinematic_region for (sym, limb) in robot.ContactPointDescriptions)))
# end

In [7]:
states, inputs, diagnostics = miqp_trajopt(robot, environment, initialstate, optparams);

In [8]:
[state.contact_point_states[:foot].pos for state in states]

10-element Array{StaticArrays.SVector{2,Float64},1}:
 [0.5,0.0]          
 [0.5,0.0]          
 [0.5,0.0]          
 [0.5,0.0]          
 [0.60728,0.0164197]
 [0.740633,0.0]     
 [0.740633,0.0]     
 [0.740633,0.0]     
 [0.740633,0.0]     
 [0.740633,0.0]     

In [ ]:
ts = collect(0 : optparams.Δt : optparams.Δt * (optparams.nsteps - 1))
brstates = BoxRobots.BoxRobotState.(states)
brinputs = BoxRobots.BoxRobotInput.(inputs)
trajectory = BoxRobots.Trajectory(ts, BoxRobots.BoxRobotSimulationData.(ts, brstates, brinputs))
playback_speed = 0.5
vis_options = BoxRobots.BoxRobotVisualizerOptions(force_arrow_normalizer=robot.m*norm(robot.g), playback_dt = optparams.Δt / playback_speed)
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window(); sleep(1)
vis = Visualizer()
# BoxRobots.draw_box_robot_state(vis, brstates[1], options = vis_options, input = brinputs[1])
BoxRobots.playback_trajectory(vis, trajectory, options = vis_options)

In [10]:
vis_options

BoxRobots.BoxRobotVisualizerOptions
  force_arrow_normalizer: Float64 588.6
  com_radius: Float64 0.1
  contact_point_radius: Float64 0.05
  playback_dt: Float64 0.05


In [11]:
inputs[end][:foot]

MixedIntegerExperiments.ContactPointInput{2,Float64}([0.0,0.0],[-63.6982,553.071])

In [12]:
constraint_violation_norm(diagnostics, Inf)

19.977229543850605

In [13]:
# using MixedIntegerExperiments
# vis = MixedIntegerExperiments.plot_piecewise_mccormick(5, -1., 1., -1., 1.);

In [14]:
# using Base.Test
# # TODO: more checks on the final result

# zvals = similar(zs, Float64)
# for i = 1 : length(contacts), j = 1 : length(steps)
#     @test sum(getvalue(zs[contacts(i), steps(j)])) == 1
# end

In [15]:
# gr()
# plt = plot_environment(environment)
# bla = plot!(plt, [NaN], [NaN], lab = "CoM", markershape = :circle)
# xvals = getvalue.(rs[:x])
# zvals = getvalue.(rs[:z])
# anim = @animate for n = 1 : nsteps
#     plt[3] = ([xvals[n]], [zvals[n]])
# end

In [16]:
# gif(anim, loop = 3, fps = round(Int64, 1 / h))